In [ ]:
'''
This script is used for interacting with OpenLCA.
Codes are developed by Ao Chen @ETH-ML.
Connect to IPC server in OpenLCA software before running. (Port: 8080)
TODO: if you're running this script in your machine for the first time, run `product_system_reader.ipynb` first!
Search for TODO in the notebook to see how to run this notebook as needed. 
'''

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})

import os
import csv
import json
import olca
import uuid
import math
import logging
import traceback
import numpy as np
import pandas as pd
from datetime import datetime, date

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [ ]:
def get_all_process(client):
    # Get all process dataset in dataframe
    Process_descriptor = client.get_descriptors(olca.Process)
    Process_list = []
    ID_list = []
    Location_list = []

    for process in Process_descriptor:
        Process_list.append(process.name)
        ID_list.append(process.id)
        Location_list.append(process.location)
    Processes_df = pd.DataFrame(list(zip(Process_list,
                                    ID_list, Location_list)),
                                columns=['name', 'id', 'location'])
    return Processes_df

In [ ]:
Processes_df = get_all_process(client)

In [ ]:
Processes_df.head()

In [ ]:
class LCA_Calculation():
    def __init__(self, Processes_df, Impact_method, Result_path, Excel_path='', Process_list_target=''):
        self.Processes_df = Processes_df
        self.Impact_method = Impact_method
        self.Result_path = Result_path
        self.Excel_path = Excel_path
        self.Process_list_target = Process_list_target

        if not os.path.exists(self.Result_path):
            os.makedirs(self.Result_path)
    
    def create_calc_product_system(self, search_process_list, Created_PS_refProcess, cached_path):
        product_system_ids = []
        product_system_names = []
        product_system_locations = []
        
        orig_json_len = len(Created_PS_refProcess)

        for j in range(len(search_process_list)):
        # for j in range(2): # for test
            # Create product systems for selected processes
            search_process_ID = search_process_list[j]['id']
            search_process_name = search_process_list[j]['name']
            search_process_location = search_process_list[j]['location']
            if search_process_ID not in Created_PS_refProcess.keys():
                print('Creating product system of: '+search_process_name+' | '+search_process_location)
                try:
                    product_system = client.create_product_system(search_process_ID, default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
                except Exception as e:
                    print('Failed to create product system of '+search_process_ID)
                    logging.error(traceback.format_exc())
                product_system_ids.append(product_system.id)
                product_system_names.append(search_process_name)
                product_system_locations.append(search_process_location)
                Created_PS_refProcess[search_process_ID] = {'Process': [search_process_name, search_process_location, search_process_ID], 'Product_system': product_system.id}
            else:
                print('Product system of '+'"'+Created_PS_refProcess[search_process_ID]['Process'][0]+' | '+Created_PS_refProcess[search_process_ID]['Process'][1]+'"'+' has been created before.')
                print('ID: '+Created_PS_refProcess[search_process_ID]['Product_system'])
                product_system_ids.append(Created_PS_refProcess[search_process_ID]['Product_system'])
                product_system_names.append(Created_PS_refProcess[search_process_ID]['Process'][0])
                product_system_locations.append(Created_PS_refProcess[search_process_ID]['Process'][1])

        product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids, product_system_locations)), 
                                columns=['name', 'id', 'location'])
        
        if len(Created_PS_refProcess) > orig_json_len:
            Created_PS_refProcess_file = open("Created_PS_refProcess.json", "w")
            Created_PS_refProcess_file = json.dump(Created_PS_refProcess, Created_PS_refProcess_file)
            # Created_PS_refProcess_file.close()

        setup = olca.CalculationSetup()
        setup.calculation_type = olca.CalculationType.SIMPLE_CALCULATION # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
        setup.impact_method = client.find(olca.ImpactMethod, self.Impact_method)
        setup.amount = 1.0
        # amount is the amount of the functional unit (fu) of the system that should be used in the calculation; unit, flow property, etc. of the fu
        # can be also defined; by default openLCA will take the settings of the reference flow of the product system

        calc_result = {}
        result_list_cache = []
        field_cache = ['GWP100a', 'location']
        result_list = []
        for j in range(len(product_system_df)):
        # for j in range(2): # for test
            key = product_system_names[j]+' | '+product_system_locations[j]+' | '+product_system_ids[j]
            print('Calculating '+key)
            try:
                setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[j])
                calc_result[key] = client.calculate(setup)
                # client.excel_export(calc_result, 'calc_result/'+product_system_locations[j]+'_'+'calc_result.xlsx')
                result = calc_result[key]
                if result.impact_results is not None:
                    for k in range(len(result.impact_results)):
                        GWP_result = result.impact_results[k]
                        if 'GWP 100a' in GWP_result.impact_category.name:
                            GWP100a_value = GWP_result.value
                    result_list_cache.append([GWP100a_value, product_system_locations[j]])
                    result_list.append([GWP100a_value, key])
                    with open(cached_path, 'w') as f:
                        write = csv.writer(f)
                        write.writerow(field_cache)
                        write.writerows(result_list_cache)
                        print('Results cached.')
                else:
                    print('No impact results: '+key)
            except Exception as e:
                print('Failed to calculate product system: '+product_system_ids[j])
                logging.error(traceback.format_exc())
            print('Remaining number of processes to be calculated: '+str(len(product_system_df)-j-1))

        try:
            result_df = pd.DataFrame(result_list, columns=['GWP100a', 'key'])
            result_df[['Activity', 'Product', 'Cutoff', 'Location', 'ps_id']] = result_df.key.str.split('|', expand=True)
            result_df = result_df.drop(columns='key')
        except Exception as e:
            result_df = ''
            print('No results are written in result_df.')
            logging.error(traceback.format_exc())

        return result_df
        
    def cal_from_excel(self, Created_PS_refProcess={}):
        result_collector = []
        if len(self.Excel_path) > 0:
            material_df = pd.read_excel(self.Excel_path, header=None)
            material_df[['Activity', 'Product', 'Cutoff']] = material_df[0].str.split('|', 2, expand=True)
            material_df['Activity'] = material_df['Activity'].str.strip()

            for i in range(len(material_df)):
            # for i in range(2): # for test
                search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if material_df['Activity'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                cached_path = self.Result_path+'/Summary_'+material_df['Activity'][i]+'_cached.csv'
                result_df = self.create_calc_product_system(search_process_list, Created_PS_refProcess, cached_path)
                if len(result_df) != 0:
                    result_df[['GWP100a', 'Location']].to_csv(self.Result_path+'/Summary_'+material_df['Activity'][i]+'.csv', index=None)
                else:
                    print('No results are written in result_df. Check log in output.')
                result_collector.append(result_df)
            return Created_PS_refProcess, result_collector
        else:
            print('Warning: Excel path is not valid.')
            return Created_PS_refProcess, result_collector

    def cal_from_list(self, Created_PS_refProcess={}):
        result_collector = []
        if len(self.Process_list_target) > 0:
            for i in range(len(self.Process_list_target['include'])):
                if self.Process_list_target['not_include'][i] == '':
                    search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if self.Process_list_target['include'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                else:
                    search_process_list = [Processes_df.iloc[p] for p in range(len(self.Processes_df)) if self.Process_list_target['include'][i] in self.Processes_df.iloc[p]['name'] and not self.Process_list_target['not_include'][i] in self.Processes_df.iloc[p]['name']] # search for processes with specific name
                cached_path = self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'_cached.csv'
                result_df = self.create_calc_product_system(search_process_list, Created_PS_refProcess, cached_path)
                if len(result_df) != 0:
                    result_df[['GWP100a', 'Location']].to_csv(self.Result_path+'/Summary_'+self.Process_list_target['include'][i]+'.csv', index=None)
                else:
                    print('No results are written in result_df. Check log in output.')
                result_collector.append(result_df)
            return Created_PS_refProcess, result_collector
        else:
            print('Warning: Process list input is not valid.')
            return Created_PS_refProcess, result_collector

In [ ]:
Impact_method = 'IPCC 2013 no LT' # TODO: change impact method as you need
Today = date.today().strftime("%d_%b_%Y") # day, month, year
Result_path = Today+'_GWP_100a' # TODO: change result path as you need

# Excel_path = ''
Excel_path = 'List_of_Materials_ecoinvent.xlsx' # optional, 
# TODO: if no Excel would be input, Excel_path = '', 
# otherwise, change Excel name (relative path, put the excel file under the same directory as this notebook)

# Process_list_target = ''
Process_list_target = {
    'include': ['electricity, high voltage, production mix'], # key words you want to include
    'not_include': [''] # key words you do not want to include
} # optional, 
# TODO: if no list would be input, Process_list_target = ''
# if you want to search different activities, put your keywords accordingly in the second, third, ... position 
# in the list and remember to include an empty element '' when there's no 'not_include' keyword.

Created_PS_refProcess = json.load(open("Created_PS_refProcess.json", "r")) 
# The read json file stores the processes that are calculated and their attached product system.
# So this file should be varied from machine to machine. If there're no product systems created before in 
# your OpenLCA database, the json file should contain an empty dictionary.

instance1 = LCA_Calculation(Processes_df, Impact_method, Result_path, Excel_path=Excel_path, Process_list_target=Process_list_target)

Created_PS_refProcess, result_collector = instance1.cal_from_excel(Created_PS_refProcess=Created_PS_refProcess) 
Created_PS_refProcess, result_collector = instance1.cal_from_list(Created_PS_refProcess=Created_PS_refProcess) 

# TODO: if you see `ERROR 500`, Failed to call method public org.openlca.ipc.RpcResponse org.openlca.ipc.handlers.Calculator.calculate(org.openlca.ipc.RpcRequest)`,
# try to increase max. memory usage in OpenLCA then restart OpenLCA and Jupyter kernel.

In [5]:
test_process = client.get(olca.Process, uid='4f582f90-d0ae-3552-be43-70c730e82af0')

In [6]:
test_exchange = test_process.exchanges[0]

In [11]:
test_input = [exchange for exchange in test_process.exchanges if exchange.input == True]

In [15]:
test_input[0].amount

0.00212170676239501

In [16]:
np.linspace()

TypeError: 'float' object cannot be interpreted as an integer

In [7]:
test_exchange.amount

1.0

In [ ]:
test_amount = []

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_exchange.amount = 0.5

In [ ]:
test_amount.append(test_exchange.amount)

In [ ]:
test_process